# Deep Learning

## Importing libraries

In [6]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

random_state = 1
np.random.seed(random_state)

## Model

In [23]:
image_size = 200, 200
inputs = keras.Input(shape=(*image_size, 3))
conv = keras.layers.Conv2D(32, 3, activation='relu')(inputs)
max_pool_2d = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv)
flatten = keras.layers.Flatten()(max_pool_2d)
dense = keras.layers.Dense(64, activation='relu')(flatten)
output = keras.layers.Dense(1, activation='sigmoid')(dense)

model = keras.Model(inputs, output)

model.compile(
    optimizer=keras.optimizers.SGD(learning_rate=0.002, momentum=0.8),
    loss=keras.losses.BinaryCrossentropy(),
    metrics=["accuracy"]
)


## Question 1

`binary crossentropy` is a loss function for binary classification problem

In [24]:
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 200, 200, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 198, 198, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 99, 99, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 313632)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │    20,072,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,073,473 (76.57 MB)

 Trainable params: 20,073,473 (76.57 MB)

 Non-trainable params: 0 (0.00 B)

## Question 2

20,073,473 total number of parameters of the model

## Generators and Training

In [25]:
batch_size = 20

train_generator = ImageDataGenerator(
    rescale=1./255
)

train_ds = train_generator.flow_from_directory(
    "data/train",
    seed=1,
    target_size=image_size,
    batch_size=batch_size,
    class_mode="binary",
    shuffle=True,
)

test_generator = ImageDataGenerator(
    rescale=1./255 
)

test_ds = train_generator.flow_from_directory(
    "data/test",
    seed=1,
    target_size=image_size,
    batch_size=batch_size,
    class_mode="binary",
    shuffle=True,
)

Found 800 images belonging to 2 classes.
Found 201 images belonging to 2 classes.


In [26]:
history_l = model.fit(
    train_ds,
    epochs=10,
    validation_data=test_ds
)

/home/marko/miniconda3/envs/ml-zoomcamp/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 24s 593ms/step - accuracy: 0.5363 - loss: 0.7136 - val_accuracy: 0.5771 - val_loss: 0.6744
Epoch 2/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 23s 574ms/step - accuracy: 0.6236 - loss: 0.6361 - val_accuracy: 0.5970 - val_loss: 0.6466
Epoch 3/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 23s 572ms/step - accuracy: 0.7004 - loss: 0.5884 - val_accuracy: 0.6517 - val_loss: 0.6196
Epoch 4/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 23s 577ms/step - accuracy: 0.7300 - loss: 0.5441 - val_accuracy: 0.6567 - val_loss: 0.6120
Epoch 5/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 23s 583ms/step - accuracy: 0.7084 - loss: 0.5491 - val_accuracy: 0.6517 - val_loss: 0.6036
Epoch 6/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 24s 588ms/step - accuracy: 0.7811 - loss: 0.4970 - val_accuracy: 0.6517 - val_loss: 0.6439
Epoch 7/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 23s 578ms/step - accuracy: 0.7378 - loss: 0.5328 - val_accuracy: 0.6816 - val_loss: 0.5876
Epoch 8/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 24s 592ms/step - accuracy: 0.7704 - loss: 0.4742 - val_accu

## Question 3

In [31]:
median = np.median(history_l.history['accuracy'])
"Median for training accuracy for all the epochs {:.2f}".format(median)

'Median for training accuracy for all the epochs 0.74'

## Qiestion 4

In [37]:
std = np.std(history_l.history['loss'])
"Standard deviation of training loss for all the epochs {:.3f}".format(std)

'Standard deviation of training loss for all the epochs 0.066'

## Data Augmentation

In [38]:
train_generator = ImageDataGenerator(
    rescale=1./255,
    rotation_range=50,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_ds = train_generator.flow_from_directory(
    "data/train",
    seed=1,
    target_size=image_size,
    batch_size=batch_size,
    class_mode="binary",
    shuffle=True,
)


Found 800 images belonging to 2 classes.


In [39]:
history_l_aug = model.fit(
    train_ds,
    epochs=10,
    validation_data=test_ds
)

/home/marko/miniconda3/envs/ml-zoomcamp/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 27s 662ms/step - accuracy: 0.6555 - loss: 0.6324 - val_accuracy: 0.6816 - val_loss: 0.5695
Epoch 2/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 41s 648ms/step - accuracy: 0.6525 - loss: 0.6026 - val_accuracy: 0.7164 - val_loss: 0.5515
Epoch 3/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 27s 663ms/step - accuracy: 0.7247 - loss: 0.5658 - val_accuracy: 0.7065 - val_loss: 0.5765
Epoch 4/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 27s 675ms/step - accuracy: 0.6738 - loss: 0.5759 - val_accuracy: 0.7114 - val_loss: 0.5596
Epoch 5/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 27s 669ms/step - accuracy: 0.6993 - loss: 0.5818 - val_accuracy: 0.7065 - val_loss: 0.5929
Epoch 6/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 27s 680ms/step - accuracy: 0.7111 - loss: 0.5606 - val_accuracy: 0.7015 - val_loss: 0.5608
Epoch 7/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 27s 678ms/step - accuracy: 0.6847 - loss: 0.5795 - val_accuracy: 0.7015 - val_loss: 0.5698
Epoch 8/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 27s 683ms/step - accuracy: 0.7033 - loss: 0.5594 - val_accu

## Question 5

In [45]:
mean = np.mean(history_l_aug.history['val_loss'])
"Mean of test loss for all the epochs {:.2f}".format(mean)

'Mean of test loss for all the epochs 0.56'

## Qiuestion 6

In [49]:
accuracy = np.mean(history_l_aug.history['val_accuracy'][5:])
"Average of test accuracy for the last 5 epochs {:.2f}".format(accuracy)

'Average of test accuracy for the last 5 epochs 0.71'